In [7]:
import os
import numpy as np
from raug.metrics import Metrics, accuracy
from raug.utils.loader import get_labels_frequency
import pandas as pd

# Especificar o caminho para a pasta "best_results"
folder_path = "best_results"
_csv_path_test = "/home/a52550/Desktop/datasets/ISIC2017/test/ISIC-2017_Test_v2_Part3_GroundTruth.csv"
true_labels = pd.read_csv(_csv_path_test).sort_values(by=['image_id'])['category']

# Inicializar as variáveis para armazenar as previsões dos modelos
ensemble_preds_best = None
ensemble_preds_last = None
num_models = 0
ser_lab_freq = get_labels_frequency(_csv_path_test, "category", "image_id")
_labels_name = ser_lab_freq.index.values #np.reshape(ser_lab_freq.index.values, (ser_lab_freq.index.values.shape[0], 1))

df_best = pd.DataFrame()
df_last = pd.DataFrame()

# Percorrer as pastas de cada experimento
for experiment_folder in os.listdir(folder_path):

    if ".csv" in experiment_folder:
        continue

    print("Experimento: ", experiment_folder)

    _metric_options_best = {
        'save_all_path': os.path.join(folder_path, experiment_folder, 'ensemble_average/best'),
        'pred_name_scores': 'predictions.csv',
    }
    _metric_options_last = {
        'save_all_path': os.path.join(folder_path, experiment_folder, 'ensemble_average/last'),
        'pred_name_scores': 'predictions.csv',
    }


    experiment_path = os.path.join(folder_path, experiment_folder)
    metrics_best = Metrics (["accuracy", "topk_accuracy", "balanced_accuracy", "conf_matrix" "plot_conf_matrix", "precision_recall_report", "auc_and_roc_curve", "auc"] , _labels_name, _metric_options_best)
    metrics_last = Metrics (["accuracy", "topk_accuracy", "balanced_accuracy", "conf_matrix" "plot_conf_matrix", "precision_recall_report", "auc_and_roc_curve", "auc"] , _labels_name, _metric_options_last)

    ensemble_preds_best = None
    ensemble_preds_last = None

    model_folders = os.listdir(experiment_path)
    
    # Cria path para o experimento atual
    if 'ensemble_average' in model_folders:
        model_folders.remove('ensemble_average')
    else:
        os.mkdir(os.path.join(folder_path, experiment_folder, 'ensemble_average'))
        os.mkdir(os.path.join(_metric_options_best['save_all_path']))
        os.mkdir(os.path.join(_metric_options_last['save_all_path']))

    try:

        # Percorrer as pastas de cada modelo
        for model_folder in model_folders:

            model_path = os.path.join(experiment_path, model_folder)

            # Carregar as previsões do modelo atual
            model_preds_path_best = os.path.join(model_path, "test_pred_best/predictions.csv")
            model_preds_path_last = os.path.join(model_path, "test_pred_last/predictions.csv")

            model_preds_best = pd.read_csv(model_preds_path_best).sort_values(by=['image'])
            model_preds_last = pd.read_csv(model_preds_path_last).sort_values(by=['image'])

            images_id = model_preds_best[['image']]
            model_preds_best = model_preds_best[['0','1','2']]
            model_preds_last = model_preds_last[['0','1','2']]

            # Adicionar as previsões do modelo atual ao ensemble
            if ensemble_preds_best is None:
                ensemble_preds_best = model_preds_best
            else:
                ensemble_preds_best += model_preds_best

            if ensemble_preds_last is None:
                ensemble_preds_last = model_preds_last
            else:
                ensemble_preds_last += model_preds_last

            num_models += 1

        # Calcular a média das previsões dos modelos
        ensemble_preds_best /= num_models
        ensemble_preds_last /= num_models

        metrics_best.update_scores(true_labels, ensemble_preds_best.to_numpy(), images_id.to_numpy())
        metrics_last.update_scores(true_labels, ensemble_preds_last.to_numpy(), images_id.to_numpy())

        metrics_best.compute_metrics()
        metrics_last.compute_metrics()

        metrics_best.save_scores()
        metrics_last.save_scores()

        folder_name = {'folder_name': experiment_folder}
        dict_best = {**folder_name, **metrics_best.metrics_values}
        dict_last = {**folder_name, **metrics_last.metrics_values}

        del dict_best['auc_and_roc_curve']
        del dict_best['precision_recall_report']

        del dict_last['auc_and_roc_curve']
        del dict_last['precision_recall_report']

        df_best = df_best.append(pd.DataFrame(dict_best, columns=dict_best.keys(), index=[0]), ignore_index=True)
        df_last = df_last.append(pd.DataFrame(dict_last, columns=dict_last.keys(), index=[0]), ignore_index=True)

    except Exception as e:
        print("Erro no experimento ", experiment_folder)
        print(e)


df_best.to_csv(os.path.join(folder_path, 'ensemble_average_best.csv'), index=False)
df_last.to_csv(os.path.join(folder_path, 'ensemble_average_last.csv'), index=False)

# Avaliar a precisão do modelo de ensemble
# true_classes_path = os.path.join(folder_path, "true_classes.csv")
# true_classes = np.loadtxt(_csv_path_test, delimiter=",", skiprows=1)

# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# accuracy = accuracy_score(true_classes, ensemble_classes)
# precision = precision_score(true_classes, ensemble_classes, average="macro")
# recall = recall_score(true_classes, ensemble_classes, average="macro")
# f1 = f1_score(true_classes, ensemble_classes, average="macro")

# # Imprimir as métricas de avaliação
# print(f"Acurácia: {accuracy:.3f}")
# print(f"Precisão: {precision:.3f}")
# print(f"Recall: {recall:.3f}")
# print(f"F1-score: {f1:.3f}")


Experimento:  dull_razor_shades_of_gray_cropped_images_folder_2
Saving the scores in best_results/dull_razor_shades_of_gray_cropped_images_folder_2/ensemble_average/best
Saving the scores in best_results/dull_razor_shades_of_gray_cropped_images_folder_2/ensemble_average/last
Experimento:  DA0_noPP
Saving the scores in best_results/DA0_noPP/ensemble_average/best
Saving the scores in best_results/DA0_noPP/ensemble_average/last
Experimento:  dull_razor
Saving the scores in best_results/dull_razor/ensemble_average/best
Saving the scores in best_results/dull_razor/ensemble_average/last
Experimento:  DA1_noPP
Saving the scores in best_results/DA1_noPP/ensemble_average/best
Saving the scores in best_results/DA1_noPP/ensemble_average/last
Experimento:  shades_of_gray_cropped_images_folder
Saving the scores in best_results/shades_of_gray_cropped_images_folder/ensemble_average/best
Saving the scores in best_results/shades_of_gray_cropped_images_folder/ensemble_average/last
Experimento:  dull_raz

/home/a52550/Desktop/masters/cnn/learning_cycle/benchmarks/isic/raug/utils/classification_metrics.py:347: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


Saving the scores in best_results/gaussian_filter_2/ensemble_average/best
Saving the scores in best_results/gaussian_filter_2/ensemble_average/last
Experimento:  CLAHE_cropped_images_folder
Erro no experimento  CLAHE_cropped_images_folder
[Errno 2] No such file or directory: 'best_results/CLAHE_cropped_images_folder/efficientnet_b6_fold-1_lrinit-0.001_batchsize-4_optimizer-SGD_maxepochs-50_DA-None_PPen-CLAHE_PPha-None_PPco-None_PPde-None_PPno-True_PPcr-cropped_images_folder_PPre-True_16794346882817037/test_pred_best/predictions.csv'
Experimento:  dull_razor_cropped_images_folder
Saving the scores in best_results/dull_razor_cropped_images_folder/ensemble_average/best
Saving the scores in best_results/dull_razor_cropped_images_folder/ensemble_average/last
Experimento:  noDA_PPcropped_2
Saving the scores in best_results/noDA_PPcropped_2/ensemble_average/best
Saving the scores in best_results/noDA_PPcropped_2/ensemble_average/last
Experimento:  DA2_noPP
Saving the scores in best_results/DA

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>